In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, BatchNormalization
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import pandas as pd
import Functions
import var
import Viz

Using plaidml.keras.backend backend.


[Link to implementation using Keras](https://engmrk.com/alexnet-implementation-using-keras/)

In [2]:
def get_alexnet(dim):
    np.random.seed(10)
    #Instantiate an empty model
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=dim, kernel_size=(11,11), strides=(4,4), padding='valid', 
                    activation = 'relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid', 
                    activation = 'relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', 
                    activation = 'relu'))

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', 
                    activation = 'relu'))

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', 
                    activation = 'relu'))
    
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid', 
                          activation = 'relu'))

    # Passing it to a Fully Connected layer
    model.add(Flatten())
    
    # 1st Fully Connected Layer
    model.add(Dense(4096, input_shape=(224*224*3,), activation = 'relu'))
    
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.4))

    # 2nd Fully Connected Layer
    model.add(Dense(4096, activation = 'relu'))
    # Add Dropout
    model.add(Dropout(0.4))

    # 3rd Fully Connected Layer
    model.add(Dense(1000, activation = 'relu'))
    # Add Dropout
    model.add(Dropout(0.4))

    # Output Layer
    model.add(Dense(3, activation = 'softmax'))


    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
x_train, x_test, y_train, y_test = Functions.get_samples('alexnet')
print(x_train.shape, x_test.shape) 
print(y_train.shape, y_test.shape)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator


augment = True 

early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=25)
model_checkpoint = ModelCheckpoint('../models/Alexnet-ModelCheckpointWeights.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 100
batch_size = 16
if var.img_type == 'grey': 
    dim = (var.alex_dimension[0], var.alex_dimension[1], 1)
else: 
    dim = (var.alex_dimension[0], var.alex_dimension[1], 3)
    
alexnet = get_alexnet(dim =dim)

if augment: 
    augmentation =ImageDataGenerator(rotation_range = 15, width_shift_range = .1, height_shift_range = .1, 
                                                           horizontal_flip = True, fill_mode = 'nearest')
    augmentation.fit(x_train)
    alexnet_history = alexnet.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
                epochs = epochs, steps_per_epoch = 8,
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)
else: 
    
    alexnet_history = alexnet.fit(x_train, y_train, batch_size = batch_size,
                epochs = epochs, 
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)


In [ ]:
print(mobile_net.summary())

### Loss and Accuracy

In [ ]:
Viz.plot_loss_accuracy(mobile_net_history)

### ROC and AUC

In [ ]:
if var.img_type == 'grey': 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 1)
else: 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
    
mobilenet = get_mobile_net(dim)
mobilenet.load_weights('../models/MobileNet-ModelCheckpointWeights.h5') #load the best weights before overfitting

Viz.plot_roc_auc(mobilenet, x_test, y_test)

### Confusion Matrix

In [ ]:
if var.img_type == 'grey': 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 1)
else: 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
    
mobilenet = get_mobile_net(dim)
mobilenet.load_weights('../models/MobileNet-ModelCheckpointWeights.h5') #load the best weights before overfitting





y_test_prob = [np.argmax(i) for i in mobilenet.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in mobilenet.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'])


In [ ]:
# assert False

### Test on Google

In [ ]:
import cv2
from lime import lime_image

In [ ]:
base_path = '../Tests/Photos'
if var.img_type == 'grey': 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 1)
else: 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
    
mobilenet = get_mobile_net(dim)
mobilenet.load_weights('../models/MobileNet-ModelCheckpointWeights.h5') #load the best weights before overfitting

for file in os.listdir(base_path): 
    if file == '.ipynb_checkpoints':
        continue
    full_path = f'{base_path}/{file}'
    img = Functions.get_image_value(full_path, var.mobilenet_dimension, var.img_type)
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
    pred = mobilenet.predict(img)[0]
    print(f'{file}\t\t{np.argmax(pred)}\t\t{pred.max()}\t\t{pred}')

In [ ]:
img = GetPickles.get_image_value('../TestImages/AR.jpg', var.mobilenet_dimension, var.img_type)

explainer = lime_image.LimeImageExplainer()

explanation = explainer.explain_instance(img, mobilenet.predict, top_labels = 5, hide_color = 0, 
                                         num_samples = 1000)

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only = False,
                                           num_features = 10, hide_rest = False)
plt.imshow(mark_boundaries(temp/2 + .5, mask))

In [ ]:
Functions.get_prediction_bounding_box('../TestImages/AR.jpg', mobilenet, var.mobilenet_dimension)

In [ ]:
assert False

In [ ]:
from tqdm import tqdm
def get_vid_prediction_bounding_box(path, model, dim): 
    vid = cv2.VideoCapture(path)
    total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    pbar = tqdm(total = total_frames, desc = f'Splitting Video Into {total_frames} Frames')
    images = [] 
    sucess =1 
    while True: 
        try:
            success, img = vid.read() 
            img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC)
            images.append(img)
            pbar.update(1)
        except: 
            break
        

    pbar.close()
    print(total_frames)
    print(np.array(images).shape)
    return np.array(images)
   



images = get_vid_prediction_bounding_box('../Tests/Videos/Pistol.mp4', mobilenet, var.mobilenet_dimension)

In [ ]:
predictions = mobilenet.predict(images)


In [ ]:
model = mobilenet
pbar = tqdm(list(zip(predictions, images)), desc= 'Getting Base Prediction and Extracting Sliding Window... Sit Back, This Will Take A While')
windows_prob = []
for pred, img in pbar: 
    category_dict = {0: 'No Weapon', 1: 'Handgun', 2: 'Rifle'}
    cat_index = np.argmax(pred)
    cat = category_dict[cat_index]
    
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()
    rects = ss.process() 

    windows = []
    locations = []
    for x, y, w,h in rects: 
        startx = x 
        starty = y 
        endx = x+w 
        endy = y+h 
        roi = img[starty:endy, startx:endx]
        roi = cv2.resize(roi, dsize =var.mobilenet_dimension, interpolation = cv2.INTER_CUBIC)
        windows.append(roi)
        locations.append((startx, starty, endx, endy))

    windows = np.array(windows)

    predictions = model.predict(windows)

    clone = img.copy()
    cat_predictions = predictions[:,cat_index]
    pred_max_idx = np.argmax(cat_predictions)
    pred_max = cat_predictions[pred_max_idx]
    
    pred_max_window = locations[pred_max_idx]
    startx, starty, endx, endy = pred_max_window
    
    windows_prob.append((cat, (startx, starty, endx, endy)))



In [ ]:
import pickle 
pickle.dump(windows_prob, open('../Pickles/WindowsPropPistol.p', 'wb'))

In [ ]:
for i in windows_prob: 
    if i[0] != 'No Weapon': 
        print(i)

In [ ]:
count = 0 
success = 1 
images = []
while success: 
    success, img = vid.read()
    orig_img = img
    orig_img = cv2.resize(img, var.mobilenet_dimension, interpolation = cv2.INTER_CUBIC)
    orig_img = orig_img.reshape(1, orig_img.shape[0], orig_img.shape[1], orig_img.shape[2])
    pred = mobilenet.predict(orig_img)[0]
    category_dict = {0: 'No Weapon', 1: 'Handgun', 2: 'Rifle'}
    cat_index = np.argmax(pred)
    cat = category_dict[cat_index]

    print(f'{cat_index}||{cat}\t\t{pred.max()}\t\t{pred}')


    break
#     cv2.imwrite(f'../Tests/VideoFrames/{count}.jpg', image)
    count += 1

In [ ]:
img = get_image_value(path, dim, var.img_type)
img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
pred = model.predict(img)[0]

category_dict = {0: 'No Weapon', 1: 'Handgun', 2: 'Rifle'}
cat_index = np.argmax(pred)
cat = category_dict[cat_index]

print(f'{path}\t\t{cat_index}||{cat}\t\t{pred.max()}\t\t{pred}')


img = cv2.imread(path)
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
rects = ss.process() 

windows = []
locations = []
for x, y, w,h in rects: 
    startx = x 
    starty = y 
    endx = x+w 
    endy = y+h 
    roi = img[starty:endy, startx:endx]
    roi = cv2.resize(roi, dsize =var.mobilenet_dimension, interpolation = cv2.INTER_CUBIC)
    windows.append(roi)
    locations.append((startx, starty, endx, endy))

windows = np.array(windows)

predictions = model.predict(windows)

clone = img.copy()
cat_predictions = predictions[:,cat_index]
pred_max_idx = np.argmax(cat_predictions)
pred_max = cat_predictions[pred_max_idx]

pred_max_window = locations[pred_max_idx]
startx, starty, endx, endy = pred_max_window
cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,255,0),2)

text = f'{cat}'
cv2.putText(clone, text, (startx, starty), cv2.FONT_HERSHEY_SIMPLEX, .45, (0,255,0),2)


cv2.imshow(f'{cat}', clone)
cv2.waitKey(0)